# Variação Paramétrica do SVM


# Passo 1: Preparando o ambiente

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

In [2]:
DATASET_ORIGINAL = "../datasets/data_normalizada.csv"
DATASET_SMOTE = "../datasets/data_smote.csv"
DATASET_UNDER = "../datasets/data_under.csv"
DATASET_TESTE = "../datasets/data_test.csv"

In [3]:
df: pd.DataFrame = pd.read_csv(DATASET_ORIGINAL)
df_smote: pd.DataFrame = pd.read_csv(DATASET_SMOTE)
df_under: pd.DataFrame = pd.read_csv(DATASET_UNDER)
df_test: pd.DataFrame = pd.read_csv(DATASET_TESTE)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6819 entries, 0 to 6818
Data columns (total 94 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   Bankrupt?                                                6819 non-null   int64  
 1   ROA(C) before interest and depreciation before interest  6819 non-null   float64
 2   ROA(A) before interest and % after tax                   6819 non-null   float64
 3   ROA(B) before interest and depreciation after tax        6819 non-null   float64
 4   Operating Gross Margin                                   6819 non-null   float64
 5   Realized Sales Gross Margin                              6819 non-null   float64
 6   Operating Profit Rate                                    6819 non-null   float64
 7   Pre-tax net Interest Rate                                6819 non-null   float64
 8   After-tax net Interest Rate 

# Passo 2: Definição do Campo de busca

Aqui é definido os parâmetros que serão testados pelo modelo. 

Segue abaixo uma breve explicação dos parâmetros testados para o SVM:

 - C (float, default=1.0): Regularização, quanto maior, mais fraca é a força de regularização
 - kernel ("linear", "poly", "rbf", "sigmoid", "precomputed" ou função, default="rbf"): kernel usado no algorítmo
 - degree (int, default=3): apenas para o kernel = poly
 - gamma ("scale", "auto" ou float, default="scale"): coeficiente do kernel para "rbf", "poly" e "sigmoid".
 - coef0 (float, default=0.0): Termo independente na função do kernel, apenas para os kernels "poly" e "sigmoid"
 - tol (float, default=1e-3): tolerância para a parada

Como há parâmetros que apenas funcionam para Kernels espeíficos, vamos separar os testes por Kernel pois o algorítmo de escolha de parâmetros será o GridSearchCV, que olha para todas as possibilidades e não faz sentido testar um modelo SVM com kernel linear com diferentes valores de degree, por exemplo.

Segue abaixo uma breve explicação dos parâmetros testados para a MLP:

 - activation ("identity", "logistic", "tanh", "relu", default="relu") : função de ativação
 - solver ("lbfgs", "sgd", "adam", default="adam"): método para alterar os pesos
 - alpha (float, default=0.0001): Força do termo de regularização
 - learning_rate ("constant", "invscaling", "adaptive", default="constant"): taxa de aprendizagem
 - learning_rate_init (float, default=0.001): valor inicial do learning rate
 - max_iter (int, default=200): número máximo de iterações, representa o número de épocas para "sgd" e "adam"
 - tol (float, default=1e-4): tolerância para a otimização
 - warm_start (bool, default=False): se True reusa a última chamada.

Caso seja observado que o melhor método é o adam, também testaramos diferentes valores de beta_1, beta_2 e epsilon.

Além disso a MLP terá valores fixos:
 - early_stopping = True
 - validation_fraction = 0.2


In [15]:
svm_linear = {
    "svm__C": [0.5, 1.0, 1.5],
    "svm__kernel": ["linear"],
    # "svm__tol": [1e-2, 1e-3, 1e-4],
    # "mlp__activation": ["identity", "logistic", "tanh", "relu"],
    "mlp__solver": ["lbfgs", "sgd", "adam"],
    "mlp__alpha": [1e-2, 1e-3, 1e-4],
    # "mlp__learning_rate": ["constant", "invscaling", "adaptive"],
    # "mlp__learning_rate_init": [1e-2, 1e-3, 1e-4],
    # "mlp__max_iter": [100, 200, 300],
    # "mlp__tol": [1e-2, 1e-3, 1e-4, 1e-5],
    "mlp__warm_start": [True, False]
}

In [16]:
svm_poly = {
    "svm__C": [0.1, 0.5, 1.0, 1.5, 2.0],
    "svm__kernel": ["poly"],
    # "svm__degree": [2, 3, 4, 5],
    # "svm__gamma": ["scale", "auto"],
    # "svm__coef0": [0.0, 0.1, 0.5, 1.0],
    # "svm__tol": [1e-2, 1e-3, 1e-4],
    # "mlp__activation": ["identity", "logistic", "tanh", "relu"],
    "mlp__solver": ["lbfgs", "sgd", "adam"],
    "mlp__alpha": [1e-2, 1e-3, 1e-4],
    # "mlp__learning_rate": ["constant", "invscaling", "adaptive"],
    # "mlp__learning_rate_init": [1e-2, 1e-3, 1e-4],
    # "mlp__max_iter": [100, 200, 300],
    # "mlp__tol": [1e-2, 1e-3, 1e-4, 1e-5],
    "mlp__warm_start": [True, False]
}

In [17]:
svm_rbf = {
    "svm__C": [0.1, 0.5, 1.0, 1.5, 2.0],
    "svm__kernel": ["rbf"],
    # "svm__gamma": ["scale", "auto"],
    # "svm__tol": [1e-2, 1e-3, 1e-4],
    # "mlp__activation": ["identity", "logistic", "tanh", "relu"],
    "mlp__solver": ["lbfgs", "sgd", "adam"],
    "mlp__alpha": [1e-2, 1e-3, 1e-4],
    # "mlp__learning_rate": ["constant", "invscaling", "adaptive"],
    # "mlp__learning_rate_init": [1e-2, 1e-3, 1e-4],
    # "mlp__max_iter": [100, 200, 300],
    # "mlp__tol": [1e-2, 1e-3, 1e-4, 1e-5],
    "mlp__warm_start": [True, False]
}

In [18]:
svm_sigmoid = {
    "svm__C": [0.1, 0.5, 1.0, 1.5, 2.0],
    "svm__kernel": ["sigmoid"],
    # "svm__gamma": ["scale", "auto"],
    # "svm__coef0": [0.0, 0.1, 0.5, 1.0],
    # "svm__tol": [1e-2, 1e-3, 1e-4],
    # "mlp__activation": ["identity", "logistic", "tanh", "relu"],
    "mlp__solver": ["lbfgs", "sgd", "adam"],
    "mlp__alpha": [1e-2, 1e-3, 1e-4],
    # "mlp__learning_rate": ["constant", "invscaling", "adaptive"],
    # "mlp__learning_rate_init": [1e-2, 1e-3, 1e-4],
    # "mlp__max_iter": [100, 200, 300],
    # "mlp__tol": [1e-2, 1e-3, 1e-4, 1e-5],
    "mlp__warm_start": [True, False]
}

# Passo 3: Definindo a Busca por Parâmetros

Nesse passo é usado o algorítmo GridSearchCV para escolher os melhores parâmetros, já que ele testa todas as possibilidades.

Para ele, foi definido:

n_jobs=-1: para utilizarmos todos os processadores possíveis e otimizar a execução;
cv=5: para fazer um KFold com 5 folds, dividindo o dataset em 5 partes;
scoring="f1": para escolher os melhores parâmetros a partir do f1, por ser um dataset desbalanceado;
error_score='raise': para propagar possíveis erros no treinamento com dado conjunto de parâmentros e parar o teste

Comparações de performance serão feitas nas próximas etapas.

In [19]:
def SVMSearch(search_params: list[list], X_train: np.ndarray, X_test:  np.ndarray, y_train: np.ndarray, y_test: np.ndarray) -> list:
    bests = []
    for params in search_params:
        
        svm_clf = SVC(probability=True, random_state=42)
        mlp_clf = MLPClassifier(random_state=42)
        
        voting_clf = VotingClassifier(estimators=[('svm', svm_clf), ('mlp', mlp_clf)], voting='soft')

        
        print(f"Buscando com os parametros {params}")
        
        grid_search = GridSearchCV(estimator=voting_clf, param_grid=params, cv=5, scoring='accuracy', n_jobs=-1)

        best = grid_search.fit(X_train, y_train)
        
        
        print(f"Melhores parâmetros: {best.best_params_}")
        print(f"Melhor f1: {best.best_score_}")
        
        print(classification_report(y_test, best.predict(X_test)))
        bests.append(best)
    
    return bests

# Passo 4: Realizando a busca para o dataset normalizado

In [20]:
X, y = df.iloc[:, 1:], df.iloc[:, 0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [21]:
bests = SVMSearch(
    [
        svm_linear,
        svm_poly,
        svm_rbf,
        svm_sigmoid
    ],
    X_train, X_test, y_train, y_test
)

Buscando com os parametros {'svm__C': [0.5, 1.0, 1.5], 'svm__kernel': ['linear'], 'mlp__solver': ['lbfgs', 'sgd', 'adam'], 'mlp__alpha': [0.01, 0.001, 0.0001], 'mlp__warm_start': [True, False]}
Melhores parâmetros: {'mlp__alpha': 0.01, 'mlp__solver': 'sgd', 'mlp__warm_start': True, 'svm__C': 0.5, 'svm__kernel': 'linear'}
Melhor f1: 0.9684671210343583
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      2637
           1       0.50      0.01      0.02        91

    accuracy                           0.97      2728
   macro avg       0.73      0.51      0.50      2728
weighted avg       0.95      0.97      0.95      2728

Buscando com os parametros {'svm__C': [0.1, 0.5, 1.0, 1.5, 2.0], 'svm__kernel': ['poly'], 'mlp__solver': ['lbfgs', 'sgd', 'adam'], 'mlp__alpha': [0.01, 0.001, 0.0001], 'mlp__warm_start': [True, False]}
Melhores parâmetros: {'mlp__alpha': 0.01, 'mlp__solver': 'sgd', 'mlp__warm_start': True, 'svm__C': 0.1, 'svm__kernel':

c:\Users\bianc\Desktop\Projeto-AM\.venv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\bianc\Desktop\Projeto-AM\.venv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\bianc\Desktop\Projeto-AM\.venv\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

Melhores parâmetros: {'mlp__alpha': 0.01, 'mlp__solver': 'sgd', 'mlp__warm_start': True, 'svm__C': 2.0, 'svm__kernel': 'rbf'}
Melhor f1: 0.9684668225010524
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      2637
           1       0.44      0.09      0.15        91

    accuracy                           0.97      2728
   macro avg       0.71      0.54      0.56      2728
weighted avg       0.95      0.97      0.95      2728

Buscando com os parametros {'svm__C': [0.1, 0.5, 1.0, 1.5, 2.0], 'svm__kernel': ['sigmoid'], 'mlp__solver': ['lbfgs', 'sgd', 'adam'], 'mlp__alpha': [0.01, 0.001, 0.0001], 'mlp__warm_start': [True, False]}
Melhores parâmetros: {'mlp__alpha': 0.01, 'mlp__solver': 'sgd', 'mlp__warm_start': True, 'svm__C': 1.0, 'svm__kernel': 'sigmoid'}
Melhor f1: 0.9696896149218887
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      2637
           1       0.53      0.10      0.17

Os modelos tiveram resultados muito parecidos, mas podemos ver que 'mlp__alpha': 0.01, 'mlp__warm_start': True e 'mlp__solver': 'sgd' foram o melhor em todos os casos, então vamos fixar eles. Além disso, o kernel "poly" teve precision e recall iguais a zero na segunda classe, então não vamos continuar os testes com ele.

In [22]:
svm_linear_ = {
    "svm__C": [0.5, 1.0, 1.5],
    "svm__kernel": ["linear"],
    "svm__tol": [1e-2, 1e-3, 1e-4],
    "mlp__activation": ["identity", "logistic", "tanh", "relu"],
    "mlp__solver": ["sgd"],
    "mlp__alpha": [1e-2],
    # "mlp__learning_rate": ["constant", "invscaling", "adaptive"],
    # "mlp__learning_rate_init": [1e-2, 1e-3, 1e-4],
    # "mlp__max_iter": [100, 200, 300],
    # "mlp__tol": [1e-2, 1e-3, 1e-4, 1e-5],
    "mlp__warm_start": [True]
}

In [23]:
svm_rbf_ = {
    "svm__C": [0.1, 0.5, 1.0, 1.5, 2.0],
    "svm__kernel": ["rbf"],
    "svm__tol": [1e-2, 1e-3, 1e-4],
    "mlp__activation": ["identity", "logistic", "tanh", "relu"],
    "mlp__solver": ["sgd"],
    "mlp__alpha": [1e-2],
    # "mlp__learning_rate": ["constant", "invscaling", "adaptive"],
    # "mlp__learning_rate_init": [1e-2, 1e-3, 1e-4],
    # "mlp__max_iter": [100, 200, 300],
    # "mlp__tol": [1e-2, 1e-3, 1e-4, 1e-5],
    "mlp__warm_start": [True]
}

In [24]:
svm_sigmoid_ = {
    "svm__C": [0.1, 0.5, 1.0, 1.5, 2.0],
    "svm__kernel": ["sigmoid"],
    "svm__coef0": [0.0, 0.1, 0.5, 1.0],
    "svm__tol": [1e-2, 1e-3, 1e-4],
    "mlp__activation": ["identity", "logistic", "tanh", "relu"],
    "mlp__solver": ["sgd"],
    "mlp__alpha": [1e-2],
    # "mlp__learning_rate": ["constant", "invscaling", "adaptive"],
    # "mlp__learning_rate_init": [1e-2, 1e-3, 1e-4],
    # "mlp__max_iter": [100, 200, 300],
    # "mlp__tol": [1e-2, 1e-3, 1e-4, 1e-5],
    "mlp__warm_start": [True]
}

In [25]:
bests = SVMSearch(
    [
        svm_linear_,
        svm_rbf_,
        svm_sigmoid_
    ],
    X_train, X_test, y_train, y_test
)

Buscando com os parametros {'svm__C': [0.5, 1.0, 1.5], 'svm__kernel': ['linear'], 'svm__tol': [0.01, 0.001, 0.0001], 'mlp__activation': ['identity', 'logistic', 'tanh', 'relu'], 'mlp__solver': ['sgd'], 'mlp__alpha': [0.01], 'mlp__warm_start': [True]}
Melhores parâmetros: {'mlp__activation': 'logistic', 'mlp__alpha': 0.01, 'mlp__solver': 'sgd', 'mlp__warm_start': True, 'svm__C': 0.5, 'svm__kernel': 'linear', 'svm__tol': 0.01}
Melhor f1: 0.9687116198118643
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      2637
           1       1.00      0.01      0.02        91

    accuracy                           0.97      2728
   macro avg       0.98      0.51      0.50      2728
weighted avg       0.97      0.97      0.95      2728

Buscando com os parametros {'svm__C': [0.1, 0.5, 1.0, 1.5, 2.0], 'svm__kernel': ['rbf'], 'svm__tol': [0.01, 0.001, 0.0001], 'mlp__activation': ['identity', 'logistic', 'tanh', 'relu'], 'mlp__solver': ['sgd'], 'mlp_

Os modelos tiveram resultados muito parecidos, mas podemos ver que 'svm__tol': 0.01 foi o melhor em todos os casos, então vamos fixar ele. Além disso, o kernel "linear" não conseguiu diferenciar bem as classes, então vamos excluir ele. Também, só tivemos dois valores para a função de ativação, vamos diminuir nosso espaço de busca para eles. Também, novamente, o melhor valor para o svm__coef0 foi 0, então vamos fixar ele.

In [26]:
svm_rbf_ = {
    "svm__C": [1.0, 1.5, 2.0],
    "svm__kernel": ["rbf"],
    "svm__tol": [1e-2],
    "mlp__activation": ["logistic", "relu"],
    "mlp__solver": ["sgd"],
    "mlp__alpha": [1e-2],
    "mlp__learning_rate": ["constant", "invscaling", "adaptive"],
    "mlp__learning_rate_init": [1e-2, 1e-3, 1e-4],
    "mlp__max_iter": [100, 200, 300],
    "mlp__tol": [1e-2, 1e-3, 1e-4],
    "mlp__warm_start": [True]
}

In [27]:
svm_sigmoid_ = {
    "svm__C": [1.0, 1.5, 2.0],
    "svm__kernel": ["sigmoid"],
    "svm__coef0": [0.0],
    "svm__tol": [1e-2],
    "mlp__activation": ["logistic", "relu"],
    "mlp__solver": ["sgd"],
    "mlp__alpha": [1e-2],
    "mlp__learning_rate": ["constant", "invscaling", "adaptive"],
    "mlp__learning_rate_init": [1e-2, 1e-3, 1e-4],
    "mlp__max_iter": [100, 200, 300],
    "mlp__tol": [1e-2, 1e-3, 1e-4],
    "mlp__warm_start": [True]
}

In [28]:
bests = SVMSearch(
    [
        svm_rbf_,
        svm_sigmoid_
    ],
    X_train, X_test, y_train, y_test
)

Buscando com os parametros {'svm__C': [1.0, 1.5, 2.0], 'svm__kernel': ['rbf'], 'svm__tol': [0.01], 'mlp__activation': ['logistic', 'relu'], 'mlp__solver': ['sgd'], 'mlp__alpha': [0.01], 'mlp__learning_rate': ['constant', 'invscaling', 'adaptive'], 'mlp__learning_rate_init': [0.01, 0.001, 0.0001], 'mlp__max_iter': [100, 200, 300], 'mlp__tol': [0.01, 0.001, 0.0001], 'mlp__warm_start': [True]}
Melhores parâmetros: {'mlp__activation': 'logistic', 'mlp__alpha': 0.01, 'mlp__learning_rate': 'invscaling', 'mlp__learning_rate_init': 0.001, 'mlp__max_iter': 100, 'mlp__solver': 'sgd', 'mlp__tol': 0.01, 'mlp__warm_start': True, 'svm__C': 2.0, 'svm__kernel': 'rbf', 'svm__tol': 0.01}
Melhor f1: 0.9701789110102068
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      2637
           1       0.43      0.03      0.06        91

    accuracy                           0.97      2728
   macro avg       0.70      0.52      0.52      2728
weighted avg       

Analisando também a classe 1, o melhor modelo é com os parâmetros {'mlp__activation': 'relu', 'mlp__alpha': 0.01, 'mlp__learning_rate': 'constant', 'mlp__learning_rate_init': 0.01, 'mlp__max_iter': 100, 'mlp__solver': 'sgd', 'mlp__tol': 0.01, 'mlp__warm_start': True, 'svm__C': 1.0, 'svm__coef0': 0.0, 'svm__kernel': 'sigmoid', 'svm__tol': 0.01}

# Passo 5: Realizando a busca para o dataset smote

In [29]:
X_train, y_train = df_smote.iloc[:, 1:], df_smote.iloc[:, 0]
X_test, y_test = df_test.iloc[:, 1:], df_test.iloc[:, 0]

In [30]:
bests = SVMSearch(
    [
        svm_linear,
        svm_poly,
        svm_rbf,
        svm_sigmoid
    ],
    X_train, X_test, y_train, y_test
)

Buscando com os parametros {'svm__C': [0.5, 1.0, 1.5], 'svm__kernel': ['linear'], 'mlp__solver': ['lbfgs', 'sgd', 'adam'], 'mlp__alpha': [0.01, 0.001, 0.0001], 'mlp__warm_start': [True, False]}
Melhores parâmetros: {'mlp__alpha': 0.01, 'mlp__solver': 'lbfgs', 'mlp__warm_start': True, 'svm__C': 0.5, 'svm__kernel': 'linear'}
Melhor f1: 0.9854869674664627
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      2637
           1       0.39      0.45      0.42        91

    accuracy                           0.96      2728
   macro avg       0.69      0.71      0.70      2728
weighted avg       0.96      0.96      0.96      2728

Buscando com os parametros {'svm__C': [0.1, 0.5, 1.0, 1.5, 2.0], 'svm__kernel': ['poly'], 'mlp__solver': ['lbfgs', 'sgd', 'adam'], 'mlp__alpha': [0.01, 0.001, 0.0001], 'mlp__warm_start': [True, False]}
Melhores parâmetros: {'mlp__alpha': 0.001, 'mlp__solver': 'adam', 'mlp__warm_start': True, 'svm__C': 0.5, 'svm__kern

Os modelos tiveram resultados muito parecidos, para próximos testes, vamos fixar 'mlp__alpha': 0.0001, 'mlp__solver': 'lbfgs', 'mlp__warm_start': True, 'svm__C': 1.0, 'svm__kernel': 'sigmoid', que foi um pouco melhor.

In [31]:
svm_sigmoid_ = {
    "svm__C": [1.0],
    "svm__kernel": ["sigmoid"],
    "svm__coef0": [0.0, 0.5, 1],
    "svm__tol": [1e-2, 1e-3, 1e-4],
    "mlp__activation": ["identity", "logistic", "tanh", "relu"],
    "mlp__solver": ["lbfgs"],
    "mlp__alpha": [1e-4],
    "mlp__learning_rate": ["constant", "invscaling", "adaptive"],
    # "mlp__learning_rate_init": [1e-2, 1e-3, 1e-4],
    # "mlp__max_iter": [100, 200, 300],
    # "mlp__tol": [1e-2, 1e-3, 1e-4],
    "mlp__warm_start": [True]
}

In [32]:
bests = SVMSearch(
    [
        svm_sigmoid_
    ],
    X_train, X_test, y_train, y_test
)

Buscando com os parametros {'svm__C': [1.0], 'svm__kernel': ['sigmoid'], 'svm__coef0': [0.0, 0.5, 1], 'svm__tol': [0.01, 0.001, 0.0001], 'mlp__activation': ['identity', 'logistic', 'tanh', 'relu'], 'mlp__solver': ['lbfgs'], 'mlp__alpha': [0.0001], 'mlp__learning_rate': ['constant', 'invscaling', 'adaptive'], 'mlp__warm_start': [True]}
Melhores parâmetros: {'mlp__activation': 'relu', 'mlp__alpha': 0.0001, 'mlp__learning_rate': 'constant', 'mlp__solver': 'lbfgs', 'mlp__warm_start': True, 'svm__C': 1.0, 'svm__coef0': 0.5, 'svm__kernel': 'sigmoid', 'svm__tol': 0.01}
Melhor f1: 0.9854868878054998
              precision    recall  f1-score   support

           0       0.98      0.97      0.98      2637
           1       0.39      0.46      0.42        91

    accuracy                           0.96      2728
   macro avg       0.69      0.72      0.70      2728
weighted avg       0.96      0.96      0.96      2728



Assim, o melhor modelo foi: {'mlp__activation': 'relu', 'mlp__alpha': 0.0001, 'mlp__learning_rate': 'constant', 'mlp__solver': 'lbfgs', 'mlp__warm_start': True, 'svm__C': 1.0, 'svm__coef0': 0.5, 'svm__kernel': 'sigmoid', 'svm__tol': 0.01}

# Passo 6: Realizando a busca para o dataset under

In [33]:
X_train, y_train = df_under.iloc[:, 1:], df_under.iloc[:, 0]
X_test, y_test = df_test.iloc[:, 1:], df_test.iloc[:, 0]

In [34]:
bests = SVMSearch(
    [
        svm_linear,
        svm_poly,
        svm_rbf,
        svm_sigmoid
    ],
    X_train, X_test, y_train, y_test
)

Buscando com os parametros {'svm__C': [0.5, 1.0, 1.5], 'svm__kernel': ['linear'], 'mlp__solver': ['lbfgs', 'sgd', 'adam'], 'mlp__alpha': [0.01, 0.001, 0.0001], 'mlp__warm_start': [True, False]}


c:\Users\bianc\Desktop\Projeto-AM\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Melhores parâmetros: {'mlp__alpha': 0.01, 'mlp__solver': 'sgd', 'mlp__warm_start': True, 'svm__C': 1.5, 'svm__kernel': 'linear'}
Melhor f1: 0.8490950226244344
              precision    recall  f1-score   support

           0       0.99      0.82      0.90      2637
           1       0.14      0.86      0.24        91

    accuracy                           0.82      2728
   macro avg       0.57      0.84      0.57      2728
weighted avg       0.97      0.82      0.88      2728

Buscando com os parametros {'svm__C': [0.1, 0.5, 1.0, 1.5, 2.0], 'svm__kernel': ['poly'], 'mlp__solver': ['lbfgs', 'sgd', 'adam'], 'mlp__alpha': [0.01, 0.001, 0.0001], 'mlp__warm_start': [True, False]}


c:\Users\bianc\Desktop\Projeto-AM\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Melhores parâmetros: {'mlp__alpha': 0.01, 'mlp__solver': 'sgd', 'mlp__warm_start': True, 'svm__C': 0.5, 'svm__kernel': 'poly'}
Melhor f1: 0.8566365007541478
              precision    recall  f1-score   support

           0       0.99      0.85      0.92      2637
           1       0.16      0.85      0.28        91

    accuracy                           0.85      2728
   macro avg       0.58      0.85      0.60      2728
weighted avg       0.97      0.85      0.90      2728

Buscando com os parametros {'svm__C': [0.1, 0.5, 1.0, 1.5, 2.0], 'svm__kernel': ['rbf'], 'mlp__solver': ['lbfgs', 'sgd', 'adam'], 'mlp__alpha': [0.01, 0.001, 0.0001], 'mlp__warm_start': [True, False]}


c:\Users\bianc\Desktop\Projeto-AM\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Melhores parâmetros: {'mlp__alpha': 0.01, 'mlp__solver': 'sgd', 'mlp__warm_start': True, 'svm__C': 0.1, 'svm__kernel': 'rbf'}
Melhor f1: 0.8450226244343891
              precision    recall  f1-score   support

           0       1.00      0.84      0.91      2637
           1       0.16      0.90      0.27        91

    accuracy                           0.84      2728
   macro avg       0.58      0.87      0.59      2728
weighted avg       0.97      0.84      0.89      2728

Buscando com os parametros {'svm__C': [0.1, 0.5, 1.0, 1.5, 2.0], 'svm__kernel': ['sigmoid'], 'mlp__solver': ['lbfgs', 'sgd', 'adam'], 'mlp__alpha': [0.01, 0.001, 0.0001], 'mlp__warm_start': [True, False]}
Melhores parâmetros: {'mlp__alpha': 0.01, 'mlp__solver': 'sgd', 'mlp__warm_start': True, 'svm__C': 1.5, 'svm__kernel': 'sigmoid'}
Melhor f1: 0.8450980392156863
              precision    recall  f1-score   support

           0       0.99      0.83      0.90      2637
           1       0.15      0.88      0.26

c:\Users\bianc\Desktop\Projeto-AM\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Os resultados dos modelos foram muito próximos, vamos fixar 'mlp__alpha': 0.01, 'mlp__solver': 'sgd', 'mlp__warm_start': True e desconsiderar o kernel linar, que teve a menor performance.

In [46]:
svm_sigmoid_ = {
    "svm__C": [0.1, 0.5, 1.0, 1.5, 2.0],
    "svm__kernel": ["sigmoid"],
    "svm__coef0": [0.0, 0.5, 1],
    "svm__tol": [1e-2, 1e-3, 1e-4],
    "mlp__activation": ["identity", "logistic", "tanh", "relu"],
    "mlp__solver": ["sgd"],
    "mlp__alpha": [1e-3],
    # "mlp__learning_rate": ["constant", "invscaling", "adaptive"],
    # "mlp__learning_rate_init": [1e-2, 1e-3, 1e-4],
    # "mlp__max_iter": [100, 200, 300],
    # "mlp__tol": [1e-2, 1e-3, 1e-4],
    "mlp__warm_start": [True]
}

In [47]:
svm_rbf_ = {
    "svm__C": [0.1, 0.5, 1.0, 1.5, 2.0],
    "svm__kernel": ["rbf"],
    "svm__tol": [1e-2, 1e-3, 1e-4],
    "mlp__activation": ["identity", "logistic", "tanh", "relu"],
    "mlp__solver": ["sgd"],
    "mlp__alpha": [1e-3],
    # "mlp__learning_rate": ["constant", "invscaling", "adaptive"],
    # "mlp__learning_rate_init": [1e-2, 1e-3, 1e-4],
    # "mlp__max_iter": [100, 200, 300],
    # "mlp__tol": [1e-2, 1e-3, 1e-4],
    "mlp__warm_start": [True]
}

In [48]:
svm_poly_ = {
    "svm__C": [0.1, 0.5, 1.0, 1.5, 2.0],
    "svm__kernel": ["poly"],
    "svm__coef0": [0.0, 0.5, 1],
    "svm__degree": [3, 4, 5],
    "svm__tol": [1e-2, 1e-3, 1e-4],
    "mlp__activation": ["identity", "logistic", "tanh", "relu"],
    "mlp__solver": ["sgd"],
    "mlp__alpha": [1e-3],
    # "mlp__learning_rate": ["constant", "invscaling", "adaptive"],
    # "mlp__learning_rate_init": [1e-2, 1e-3, 1e-4],
    # "mlp__max_iter": [100, 200, 300],
    # "mlp__tol": [1e-2, 1e-3, 1e-4],
    "mlp__warm_start": [True]
}

In [49]:
bests = SVMSearch(
    [
        svm_poly_,
        svm_rbf_,
        svm_sigmoid_
    ],
    X_train, X_test, y_train, y_test
)

Buscando com os parametros {'svm__C': [0.1, 0.5, 1.0, 1.5, 2.0], 'svm__kernel': ['poly'], 'svm__coef0': [0.0, 0.5, 1], 'svm__degree': [3, 4, 5], 'svm__tol': [0.01, 0.001, 0.0001], 'mlp__activation': ['identity', 'logistic', 'tanh', 'relu'], 'mlp__solver': ['sgd'], 'mlp__alpha': [0.001], 'mlp__warm_start': [True]}


c:\Users\bianc\Desktop\Projeto-AM\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Melhores parâmetros: {'mlp__activation': 'relu', 'mlp__alpha': 0.001, 'mlp__solver': 'sgd', 'mlp__warm_start': True, 'svm__C': 0.5, 'svm__coef0': 0.0, 'svm__degree': 3, 'svm__kernel': 'poly', 'svm__tol': 0.01}
Melhor f1: 0.8566365007541478
              precision    recall  f1-score   support

           0       0.99      0.85      0.92      2637
           1       0.16      0.85      0.28        91

    accuracy                           0.85      2728
   macro avg       0.58      0.85      0.60      2728
weighted avg       0.97      0.85      0.90      2728

Buscando com os parametros {'svm__C': [0.1, 0.5, 1.0, 1.5, 2.0], 'svm__kernel': ['rbf'], 'svm__tol': [0.01, 0.001, 0.0001], 'mlp__activation': ['identity', 'logistic', 'tanh', 'relu'], 'mlp__solver': ['sgd'], 'mlp__alpha': [0.001], 'mlp__warm_start': [True]}


c:\Users\bianc\Desktop\Projeto-AM\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Melhores parâmetros: {'mlp__activation': 'logistic', 'mlp__alpha': 0.001, 'mlp__solver': 'sgd', 'mlp__warm_start': True, 'svm__C': 0.5, 'svm__kernel': 'rbf', 'svm__tol': 0.01}
Melhor f1: 0.8565610859728506
              precision    recall  f1-score   support

           0       1.00      0.83      0.90      2637
           1       0.15      0.90      0.26        91

    accuracy                           0.83      2728
   macro avg       0.57      0.86      0.58      2728
weighted avg       0.97      0.83      0.88      2728

Buscando com os parametros {'svm__C': [0.1, 0.5, 1.0, 1.5, 2.0], 'svm__kernel': ['sigmoid'], 'svm__coef0': [0.0, 0.5, 1], 'svm__tol': [0.01, 0.001, 0.0001], 'mlp__activation': ['identity', 'logistic', 'tanh', 'relu'], 'mlp__solver': ['sgd'], 'mlp__alpha': [0.001], 'mlp__warm_start': [True]}
Melhores parâmetros: {'mlp__activation': 'logistic', 'mlp__alpha': 0.001, 'mlp__solver': 'sgd', 'mlp__warm_start': True, 'svm__C': 0.5, 'svm__coef0': 0.0, 'svm__kernel': 'sigm

c:\Users\bianc\Desktop\Projeto-AM\.venv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


O melhor modelo foi com os parâmetros {'mlp__activation': 'relu', 'mlp__alpha': 0.001, 'mlp__solver': 'sgd', 'mlp__warm_start': True, 'svm__C': 0.5, 'svm__coef0': 0.0, 'svm__degree': 3, 'svm__kernel': 'poly', 'svm__tol': 0.01}